# Reto 10


Cargando las credenciales de Twitter


In [0]:
#!pip install dnspython
#!pip install pymongo

In [0]:
import ____, keys_DB

In [0]:
auth = tweepy.OAuthHandler(
    keys_DB.consumer_key, keys_DB.consumer_secret)
auth.set_access_token(keys_DB.access_token, 
    keys_DB.access_token_secret)

In [0]:
api = tweepy.API(auth, wait_on_rate_limit=True, 
                 wait_on_rate_limit_notify=True)               

Cargando los datos de los senadores de EUA


In [0]:
import ____ as ____

In [0]:
senators_df = pd.read_csv('senators.csv')

In [0]:
senators_df['TwitterID'] = senators_df['TwitterID'].astype(str)

In [0]:
pd.options.display.max_columns = 6

In [0]:
senators_df.head()

Configurando MongoClient

In [0]:
from ____ import ____

atlas_client = MongoClient(keys_DB.mongo_connection_string)
db = atlas_client.____

Configurando TweetListener

In [0]:
from tweetlistenerdb import TweetListener

tweet_limit = 1000
twitter_stream = tweepy.Stream(api.auth, 
    TweetListener(api, db, tweet_limit))
twitter_stream.filter(track=senators_df.TwitterHandle.tolist(),
    follow=senators_df.TwitterID.tolist())

Contando los Tweets


In [0]:
db.tweets.create_index([('$**', 'text')])
tweet_counts = []

for senator in senators_df.TwitterHandle:
    tweet_counts.append(db.tweets.count_documents(
        {"$text": {"$search": senator}}))


tweet_counts_df = senators_df.assign(Tweets=tweet_counts)
tweet_counts_df.sort_values(by='Tweets', 
    ascending=False).head(10)    

Obteniendo las localizaciones

In [0]:
from geopy import OpenMapQuest
import time
from state_codes import state_codes

In [0]:
geo = OpenMapQuest(api_key=keys_DB.mapquest_key) 
states = tweet_counts_df.State.unique()
states.sort()
locations = []
for state in states:
    processed = False
    delay = .1 
    while not processed:
        try:
            locations.append(
                geo.geocode(state_codes[state] + ', USA'))
            print(locations[-1])  
            processed = True
        except:  # timed out, so wait before trying again
            print('OpenMapQuest service timed out. Waiting.')
            time.sleep(delay)
            delay += .1

Agrupando los Tweets por estados

In [0]:
tweets_counts_by_state = tweet_counts_df.groupby(
    'State', as_index=False).sum()
tweets_counts_by_state.head()

Creando el mapa

In [0]:
import folium

In [0]:
usmap = folium.Map(location=[39.8283, -98.5795], 
                   zoom_start=4, detect_retina=True,
                   tiles='Stamen Toner')

In [0]:
 choropleth = folium.Choropleth(
    geo_data='us-states.json',
    name='choropleth',
    data=tweets_counts_by_state,
    columns=['State', 'Tweets'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Tweets by State'
).add_to(usmap)

In [0]:
layer = folium.LayerControl().add_to(usmap)

Creando los marcadores de cada estado

In [0]:
sorted_df = tweet_counts_df.sort_values(
    by='Tweets', ascending=False)

In [0]:
for index, (name, group) in enumerate(sorted_df.groupby('State')):
    strings = [state_codes[name]]  # used to assemble popup text

    for s in group.itertuples():
        strings.append(
            f'{s.Name} ({s.Party}); Tweets: {s.Tweets}')
        
    text = '<br>'.join(strings)  
    marker = folium.Marker(
        (locations[index].latitude, locations[index].longitude), 
        popup=text)
    marker.add_to(usmap) 

Mostrando el mapa

In [0]:
usmap